<a href="https://colab.research.google.com/github/kotharisanjana/CMPE258_DeepLearning_Spring2023/blob/main/Assignment_5/BabyAGI_Langchain/BabyAGI_with_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installation
!pip install langchain openai tiktoken faiss-cpu google-search-results  -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.8/548.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 48.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.9 MB/s eta 0:00:

In [ ]:
# Imports
import os
from collections import deque
from typing import Dict, List, Optional, Any
import faiss

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores.faiss import FAISS
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain

In [ ]:
os.environ['SERPAPI_API_KEY'] = ''
os.environ['OPENAI_API_KEY'] = ''

In [ ]:
# Define embedding model
embeddings_model = OpenAIEmbeddings()

# Initialize the vectorstore as empty
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks

In [ ]:
# To select new tasks to add to the list

class TaskCreationChain(LLMChain):
    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        task_creation_template = (
            'You are a task creation AI that uses the result of an execution agent'
            ' to create new tasks with the following objective: {objective},'
            ' The last completed task has the result: {result}.'
            ' This result was based on this task description: {task_description}.'
            ' These are incomplete tasks: {incomplete_tasks}.'
            ' Based on the result, create new tasks to be completed'
            ' by the AI system that do not overlap with incomplete tasks.'
            ' Return the tasks as an array.'
        )
        prompt = PromptTemplate(
            template=task_creation_template,
            input_variables=['result', 'task_description', 'incomplete_tasks', 'objective'],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

In [ ]:
# Reprioritize tasks

class TaskPrioritizationChain(LLMChain):
    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        task_prioritization_template = (
            'You are a task prioritization AI tasked with cleaning the formatting of and reprioritizing'
            ' the following tasks: {task_names}.'
            ' Consider the ultimate objective of your team: {objective}.'
            ' Do not remove any tasks. Return the result as a numbered list, like:'
            ' #. First task'
            ' #. Second task'
            ' Start the task list with number {next_task_id}.'
        )
        prompt = PromptTemplate(
            template=task_prioritization_template,
            input_variables=['task_names', 'next_task_id', 'objective'],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

In [ ]:
todo_prompt = PromptTemplate.from_template('You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}')
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name = 'Search',
        func=search.run,
        description='useful for when you need to answer questions about current events'
    ),
    Tool(
        name = 'TODO',
        func=todo_chain.run,
        description='useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!'
    )
]

prefix = '''You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}.'''
suffix = '''Question: {task}
{agent_scratchpad}'''
prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=['objective', 'task', 'context','agent_scratchpad']
)

In [ ]:
def get_next_task(task_creation_chain: LLMChain, result: Dict, task_description: str, task_list: List[str], objective: str) -> List[Dict]:
    incomplete_tasks = ', '.join(task_list)
    response = task_creation_chain.run(result=result, task_description=task_description, incomplete_tasks=incomplete_tasks, objective=objective)
    new_tasks = response.split('\n')
    return [{'task_name': task_name} for task_name in new_tasks if task_name.strip()]

In [ ]:
def prioritize_tasks(task_prioritization_chain: LLMChain, this_task_id: int, task_list: List[Dict], objective: str) -> List[Dict]:
    task_names = [t['task_name'] for t in task_list]
    next_task_id = int(this_task_id) + 1
    response = task_prioritization_chain.run(task_names=task_names, next_task_id=next_task_id, objective=objective)
    new_tasks = response.split('\n')
    prioritized_task_list = []
    for task_string in new_tasks:
        if not task_string.strip():
            continue
        task_parts = task_string.strip().split('.', 1)
        if len(task_parts) == 2:
            task_id = task_parts[0].strip()
            task_name = task_parts[1].strip()
            prioritized_task_list.append({'task_id': task_id, 'task_name': task_name})
    return prioritized_task_list

In [ ]:
def _get_top_tasks(vectorstore, query: str, k: int) -> List[str]:
    results = vectorstore.similarity_search_with_score(query, k=k)
    if not results:
        return []
    sorted_results, _ = zip(*sorted(results, key=lambda x: x[1], reverse=True))
    return [str(item.metadata['task']) for item in sorted_results]

def execute_task(vectorstore, execution_chain: LLMChain, objective: str, task: str, k: int = 5) -> str:
    context = _get_top_tasks(vectorstore, query=objective, k=k)
    return execution_chain.run(objective=objective, context=context, task=task)

In [ ]:
# BabyAGI controller

class BabyAGI(Chain, BaseModel):

    task_list: deque = Field(default_factory=deque)
    task_creation_chain: TaskCreationChain = Field(...)
    task_prioritization_chain: TaskPrioritizationChain = Field(...)
    execution_chain: AgentExecutor = Field(...)
    task_id_counter: int = Field(1)
    vectorstore: VectorStore = Field(init=False)
    max_iterations: Optional[int] = None
        
    class Config:
        arbitrary_types_allowed = True

    def add_task(self, task: Dict):
        self.task_list.append(task)

    def print_task_list(self):
        print('\033[95m\033[1m' + '\n*****TASK LIST*****\n' + '\033[0m\033[0m')
        for t in self.task_list:
            print(str(t['task_id']) + ': ' + t['task_name'])

    def print_next_task(self, task: Dict):
        print('\033[92m\033[1m' + '\n*****NEXT TASK*****\n' + '\033[0m\033[0m')
        print(str(task['task_id']) + ': ' + task['task_name'])

    def print_task_result(self, result: str):
        print('\033[93m\033[1m' + '\n*****TASK RESULT*****\n' + '\033[0m\033[0m')
        print(result)
        
    @property
    def input_keys(self) -> List[str]:
        return ['objective']
    
    @property
    def output_keys(self) -> List[str]:
        return []

    def _call(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        objective = inputs['objective']
        first_task = inputs.get('first_task', 'Make a todo list')
        self.add_task({'task_id': 1, 'task_name': first_task})
        num_iters = 0
        while True:
            if self.task_list:
                self.print_task_list()

                # Step 1: Pull the first task
                task = self.task_list.popleft()
                self.print_next_task(task)

                # Step 2: Execute the task
                result = execute_task(
                    self.vectorstore, self.execution_chain, objective, task['task_name']
                )
                this_task_id = int(task['task_id'])
                self.print_task_result(result)

                # Step 3: Store the result in Pinecone
                result_id = f"result_{task['task_id']}"
                self.vectorstore.add_texts(
                    texts=[result],
                    metadatas=[{'task': task['task_name']}],
                    ids=[result_id],
                )

                # Step 4: Create new tasks and reprioritize task list
                new_tasks = get_next_task(
                    self.task_creation_chain, result, task['task_name'], [t['task_name'] for t in self.task_list], objective
                )
                for new_task in new_tasks:
                    self.task_id_counter += 1
                    new_task.update({'task_id': self.task_id_counter})
                    self.add_task(new_task)
                self.task_list = deque(
                    prioritize_tasks(
                        self.task_prioritization_chain, this_task_id, list(self.task_list), objective
                    )
                )
            num_iters += 1
            if self.max_iterations is not None and num_iters == self.max_iterations:
                print('\033[91m\033[1m' + '\n*****TASK ENDING*****\n' + '\033[0m\033[0m')
                break
        return {}

    @classmethod
    def from_llm(
        cls,
        llm: BaseLLM,
        vectorstore: VectorStore,
        verbose: bool = False,
        **kwargs
    ) -> 'BabyAGI':
        '''Initialize the BabyAGI Controller.'''
        task_creation_chain = TaskCreationChain.from_llm(
            llm, verbose=verbose
        )
        task_prioritization_chain = TaskPrioritizationChain.from_llm(
            llm, verbose=verbose
        )
        llm_chain = LLMChain(llm=llm, prompt=prompt)
        tool_names = [tool.name for tool in tools]
        agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
        agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
        return cls(
            task_creation_chain=task_creation_chain,
            task_prioritization_chain=task_prioritization_chain,
            execution_chain=agent_executor,
            vectorstore=vectorstore,
            **kwargs
        )

In [ ]:
OBJECTIVE = 'Report news about global warming and the current prevailing effects in different places on the planet'

In [ ]:
# Setting temperature to zero so that model does not hallucinate
llm = OpenAI(temperature=0)

In [ ]:
verbose=False

max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    verbose=verbose,
    max_iterations=max_iterations
)

In [ ]:
baby_agi({'objective': OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: What tasks should I complete to make a todo list?
Action: TODO
Action Input: Report news about global warming and the current prevailing effects in different places on the planet.
Observation: 

Todo List:

1. Research current news and information about global warming and its effects.

2. Identify different places on the planet that are being affected by global warming.

3. Collect data and statistics about the effects of global warming in each of the identified places.

4. Analyze the data and statistics to identify trends and patterns.

5. Write a report summarizing the findings and conclusions.

6. Publish the report in a suitable format (e.g. online, print, etc.).

7. Promote the report through various channels (e.g. social media, press releases, etc.).
Thought: I now know the final answer
Final Answer: The final answer is to create a todo list to rep

{'objective': 'Report news about global warming and the current prevailing effects in different places on the planet'}